<a href="https://colab.research.google.com/github/sbehmanesh/DecisionTree_AI/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
import pandas as pd
import math
import numpy as np
import random

tree_depth = 4
learn_ratio = 0.7
total_tree_number = 100
min_col_number_ratio = 0.5

In [22]:
"""حال تابعی می نویسیم که آنتروپی را محاسبه کند"""

def ent(target_labels,col_label,col_values,target_data):
  ent = 0
  selected_target_leaf = False
  nparray = np.array(col_values)
  target_to_search_row_indexes = np.where(nparray == col_label)[0]
  taget_data_for_label = np.asarray(target_data)[target_to_search_row_indexes].tolist() 
  if len(taget_data_for_label) > 0:
    for target in target_labels:
      ratio = taget_data_for_label.count(target)/len(taget_data_for_label)
      if ratio > 0:
        ent += -ratio*math.log2(ratio)
      else :
        selected_target_leaf = target

  return ent , len(taget_data_for_label) , selected_target_leaf


"""با استفاده از تابع بالا تابع بهره اطلاعات را بدست می آوریم"""

def info_gain(parent_entropy,target_labels,col_labels,col_data,target_data):
  gain = parent_entropy
  for label in col_labels:
    entro , data_number , leaf = ent(target_labels,label,list(col_data),list(target_data))
    gain -= (data_number/len(target_data))*entro
  return gain


def recursive_id3(root,labels,column_names,learning_data,target_col,input_target_to_search_row_indexes,final_tree,already_used_cols,tree_depth,depth = 0):
    if depth > tree_depth:
      return 0

    branches = labels[root]

    for branch in branches:
      # print("1. branch : ",branch)
      nparray = np.array(list(learning_data[root]))
      target_to_search_row_indexes = np.where(nparray == branch)[0]
      # print("2. indexes : ",target_to_search_row_indexes)
      target_to_search_row_indexes = np.intersect1d(target_to_search_row_indexes,input_target_to_search_row_indexes)
      # print("3. input indexes : ",input_target_to_search_row_indexes)
      # print("4. common indexes : ",target_to_search_row_indexes)
      target_data_for_label = np.asarray(list(learning_data[target_col]))[target_to_search_row_indexes].tolist()
      col_data_for_label    = np.asarray(list(learning_data[root]))[target_to_search_row_indexes].tolist()
      
      if depth == tree_depth:
        # We should now calculate which target label has more popularity
        if len(target_data_for_label): 
          most_popular_target = max(set(target_data_for_label), key = target_data_for_label.count)
          final_tree[root][branch] = most_popular_target
          # print("Last depth leaf : ",most_popular_target)
      else:
        entropy = 0
        leaf_type = -1
        total_count = len(list(target_data_for_label))
        # print("total : ",total_count)
        for label in labels[target_col]:
          repeat_num = list(target_data_for_label).count(label)
          if repeat_num > 0:
            entropy  += -(repeat_num/total_count)*math.log2(repeat_num/total_count)
            leaf_type = label

        # print("entropy : ",entropy)
        if entropy == 0:  
          # is leaf
          # print(depth,") label -> ",branch," is leaf : ",leaf_type)
          if leaf_type != -1 :
            final_tree[root][branch] = leaf_type
        else:
          gains = {}
          for col in column_names[0:-1]:
            if col in already_used_cols.keys():
              if depth not in already_used_cols[col] and len(already_used_cols[col]) > 0:
                continue
            gain = info_gain(entropy,labels[target_col],labels[col],col_data_for_label,target_data_for_label)
            gains[col] = gain
            # print(gains)

          if len(gains.keys()) > 0:
            sorted_gains = sorted(gains.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
            # print(sorted_gains)
            # print(depth,") label -> ",branch," node : ",sorted_gains[-1][0]," : ",sorted_gains[-1][1])
            if sorted_gains[-1][0] in already_used_cols.keys():
              already_used_cols[sorted_gains[-1][0]].append(depth)
            else:
              already_used_cols[sorted_gains[-1][0]] = [depth]

            final_tree[root][branch] = {sorted_gains[-1][0]:{}}
            recursive_id3(sorted_gains[-1][0],labels,column_names,learning_data,target_col,target_to_search_row_indexes.copy(),final_tree[root][branch],already_used_cols,tree_depth,depth + 1)
          else: 
            print(depth,") Tree ended here")

بارگذاری داد ها و تقسیم بندی برای آموزش درخت های تصمیم متفاوت

In [23]:
drive.mount('/content/drive')
raw_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prison_dataset.csv')
column_names = raw_data.columns
labels = {}
target_col = column_names[-1]
for column_name in column_names:
  labels[column_name] = list(set(raw_data[column_name].values))

print("labels : ",labels)

total_row_number = len(list(raw_data[target_col].values))
learning_data_count = math.floor(learn_ratio*total_row_number)
test_data_count = total_row_number - learning_data_count
learning_data_indexes = random.sample(range(total_row_number), learning_data_count)
test_data_indexes = []

print("total row number is ",total_row_number)

for i in range(total_row_number):
  if i not in learning_data_indexes:
    test_data_indexes.append(i)

learning_data = {}
test_data = {}
for col in column_names:
  learning_data[col] = np.asarray(list(raw_data[col].values))[learning_data_indexes].tolist()
  test_data[col] = np.asarray(list(raw_data[col].values))[test_data_indexes].tolist()


trees_learning_data = []
for i in range(total_tree_number):
  sub_total_row_number = len(list(learning_data[target_col]))
  sub_learning_data_count = math.floor(learn_ratio*sub_total_row_number)
  sub_learning_data_indexes = random.sample(range(sub_total_row_number), sub_learning_data_count)
  sub_learning_data = {}
  random_col_indexes = random.sample(range(len(list(column_names))-1),random.randint(math.floor(min_col_number_ratio*(len(list(column_names))-1)),len(list(column_names))-1))
  col_index_checker = 0
  for col in column_names:
    if col_index_checker in random_col_indexes and col != target_col:
      sub_learning_data[col] = np.asarray(list(learning_data[col]))[sub_learning_data_indexes].tolist()
      if 'random_column_names' in sub_learning_data.keys():
        sub_learning_data['random_column_names'].append(col)
      else:
        sub_learning_data['random_column_names'] = [col]
    col_index_checker += 1

  sub_learning_data[target_col] = np.asarray(list(learning_data[target_col]))[sub_learning_data_indexes].tolist()




  trees_learning_data.append(sub_learning_data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
labels :  {'Fiscal Year Released': [2010, 2013, 2015], 'Recidivism Reporting Year': [2016, 2018, 2013], 'Race - Ethnicity': ['White', 'Black'], 'Age At Release': ['>45', '<45'], 'Convicting Offense Classification': ['Other Felony', 'D Felony'], 'Convicting Offense Type': ['Drug', 'Other', 'Violent'], 'Convicting Offense Subtype': ['Trafficking', 'Other'], 'Main Supervising District': ['3JD', '5JD'], 'Release Type': ['Discharged End of Sentence', 'Parole'], 'Part of Target Population': ['No', 'Yes'], 'Recidivism - Return to Prison numeric': [0, 1]}
total row number is  15424


ابتدا پارامتر های مورد نیاز را مشخص می کنیم

In [24]:
def create_id3_tree(tree_depth , column_names , labels , learning_data):
  """حال باید آنتروپی را محاسبه کنیم"""
  final_tree = {}
  entropy = 0
  target_col = column_names[-1]
  total_count = len(list(learning_data[target_col]))
  for label in labels[target_col]:
    repeat_num = list(learning_data[target_col]).count(label)
    if repeat_num > 0:
      entropy += -(repeat_num/total_count)*math.log2(repeat_num/total_count)

  """حال می توانیم بهره اطلاعات را محسابه کنیم"""

  gains = {}
  for col in learning_data['random_column_names']:
    gain = info_gain(entropy,labels[target_col],labels[col],list(learning_data[col]),list(learning_data[target_col]))
    gains[col] = gain

  sorted_gains = sorted(gains.items(), key = lambda kv:(kv[1], kv[0]))
  root = sorted_gains[-1][0]
  # print("root => ",root," : gain = ",sorted_gains[-1][1])

  """حال برای هر زیر شاخه می خواهیم گره بعدی را مشخص کنیم"""

  column_names = list(learning_data['random_column_names'])
  already_used_cols = {root:[0]}
  final_tree[root] = {}
  target_to_search_row_indexes = list(range(0,len(list(learning_data[root]))))
  recursive_id3(root,labels,column_names.copy(),learning_data,target_col,target_to_search_row_indexes.copy(),final_tree,already_used_cols,tree_depth,1)
  
  return final_tree

In [25]:
total_final_trees = []
repeated_trees = 1
for i in range(total_tree_number):
  learn_data = trees_learning_data[i]
  print(learn_data['random_column_names'])
  new_final_tree = create_id3_tree(tree_depth,column_names.copy(),labels.copy(),learn_data)
  if new_final_tree in total_final_trees:
    repeated_trees += 1
  else:
    total_final_trees.append(new_final_tree)
    print(new_final_tree)

print(repeated_trees," were repeated from ",total_tree_number," trees")
print(total_tree_number-repeated_trees," unique trees found")




['Fiscal Year Released', 'Recidivism Reporting Year', 'Race - Ethnicity', 'Age At Release', 'Convicting Offense Classification', 'Convicting Offense Type', 'Convicting Offense Subtype', 'Release Type']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


{'Recidivism Reporting Year': {2016: {'Age At Release': {'>45': {'Convicting Offense Classification': {'Other Felony': {'Convicting Offense Subtype': {'Trafficking': 1, 'Other': 1}}, 'D Felony': {'Convicting Offense Subtype': {'Trafficking': 1, 'Other': 1}}}}, '<45': {'Convicting Offense Classification': {'Other Felony': {'Convicting Offense Subtype': {'Trafficking': 1, 'Other': 1}}, 'D Felony': {'Convicting Offense Subtype': {'Trafficking': 1, 'Other': 1}}}}}}, 2018: {'Age At Release': {'>45': {'Convicting Offense Classification': {'Other Felony': {'Convicting Offense Subtype': {'Trafficking': 0, 'Other': 0}}, 'D Felony': {'Convicting Offense Subtype': {'Trafficking': 0, 'Other': 0}}}}, '<45': {'Convicting Offense Classification': {'Other Felony': {'Convicting Offense Subtype': {'Trafficking': 0, 'Other': 0}}, 'D Felony': {'Convicting Offense Subtype': {'Trafficking': 0, 'Other': 0}}}}}}, 2013: 1}}
['Fiscal Year Released', 'Recidivism Reporting Year', 'Convicting Offense Classificatio

خروجی گرفتن از جنگل تصادفی با میانگین گیری

In [26]:
def recursive_validate_result_by_tree(tree,test_data,test_data_index,raw_test_data):
  if tree not in [0,1]:
    tree_keys = list(tree.keys())
    if len(tree_keys) > 1:
      # branches level
      for key in tree_keys:
        if key == test_data[test_data_index]:
          # print("leaf : ",key)
          return recursive_validate_result_by_tree(tree[key],[],test_data_index,raw_test_data)
    elif len(tree_keys) == 1:
      # node level
      # print("node : ",tree[tree_keys[0]])
      if tree_keys[0] in raw_test_data.keys():
        return recursive_validate_result_by_tree(tree[tree_keys[0]],raw_test_data[tree_keys[0]],test_data_index,raw_test_data)
      return "not trained"
  else:
    # last depth of tree
    # print(tree)
    return tree

pt = 0
pf = 0
nt = 0
nf = 0
not_trained = 0
for i in range(test_data_count):
  results = []
  for j in range(total_tree_number-repeated_trees):  
    tree_output= recursive_validate_result_by_tree(total_final_trees[j],[],i,test_data)
    results.append(tree_output)
    # print(j,")",tree_output)
  
  if results.count("not trained") == len(results):
    not_trained += 1
  else:
    while "not trained" in results:
      results.remove("not trained")

    if len(results) > 0:
      result = max(set(results), key=results.count)
      if result == test_data[target_col][i]:
        if result > 0:
          pt += 1
        else:
          nt += 1
      elif result != test_data[target_col][i]:
        if result is not None:
          if result > 0:
            pf += 1
        else:
          nf += 1
  

نمایش نتایج نهایی

In [27]:
print("Confusion Matrix")
print("                 real true   real false")
print("                ------------------------")
print("predicted true | ",pt,"        ",pf)
print("predicted false| ",nf,"        ",nt)
print("                ------------------------")
accuracy = ((pt + nt)/(pt+nt+pf+nf+not_trained))*100
print("accuracy : ",accuracy,"%")
print("data could not be validated : ",not_trained)

Confusion Matrix
                 real true   real false
                ------------------------
predicted true |  1627          303
predicted false|  0          1677
                ------------------------
accuracy :  91.59966731355698 %
data could not be validated :  0
